In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import re
from matplotlib import pyplot as plt


In [10]:
df = pd.read_csv('cleaned-data/train-fixed.no-ol.csv')
df.head()

,mr,ref,fixed,orig_mr
0,"name[The Eagle], eatType[coffee shop], food[Ja...",The Eagle is a low rated coffee shop near Burg...,0,"name[The Eagle], eatType[coffee shop], food[Ja..."
1,"name[The Mill], eatType[coffee shop], food[Fre...",Located near The Sorrento is a French Theme ea...,1,"name[The Mill], eatType[coffee shop], food[Fre..."
2,"name[Loch Fyne], food[French], area[riverside]...","For luxurious French food, the Loch Fyne is lo...",1,"name[Loch Fyne], food[French], customer rating..."
3,"name[The Rice Boat], eatType[restaurant], food...",The Rice Boat is an adult French restaurant wi...,1,"name[The Rice Boat], food[French], customer ra..."
4,"name[The Wrestlers], eatType[coffee shop], foo...",The Wrestlers coffee shop serves Japanese food...,0,"name[The Wrestlers], eatType[coffee shop], foo..."


In [11]:
train_data = tf.data.experimental.CsvDataset(filenames='cleaned-data/train-fixed.no-ol.csv', 
                                record_defaults=[tf.string, tf.string],
                                header=True,
                                select_cols=[0, 1])

In [12]:
for i, d in enumerate(train_data):
    if i > 1:
        break
    print(d)

(<tf.Tensor: shape=(), dtype=string, numpy=b'name[The Eagle], eatType[coffee shop], food[Japanese], priceRange[less than \xc2\xa320], customer rating[low], area[riverside], familyFriendly[yes], near[Burger King]'>, <tf.Tensor: shape=(), dtype=string, numpy=b'The Eagle is a low rated coffee shop near Burger King and the riverside that is family friendly and is less than \xc2\xa320 for Japanese food.'>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'name[The Mill], eatType[coffee shop], food[French], area[riverside], near[The Sorrento]'>, <tf.Tensor: shape=(), dtype=string, numpy=b'Located near The Sorrento is a French Theme eatery and coffee shop called The Mill, with a price range at \xc2\xa320-\xc2\xa325 it is in the riverside area.'>)


In [13]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("t5-small")

special_tokens = {'additional_special_tokens': ['<area>', '<eatType>', '<food>', '<near>',                                                      '<name>', '<customer rating>', '<priceRange>',                                                   '<familyFriendly>', '<notfamilyFriendly>',                                                       '<cr_slot>', '<pr_slot>', '<sos>']}
tokenizer.add_special_tokens(special_tokens)

12

In [2]:
from transformers.transformer import Transformer

sample_transformer = Transformer(
    num_layers=2, d_model=512, num_heads=8, dff=2048, 
    input_vocab_size=8500, target_vocab_size=8000, 
    pe_input=10000, pe_target=6000)

temp_input = tf.random.uniform((64, 38), dtype=tf.int64, minval=0, maxval=200)
temp_target = tf.random.uniform((64, 36), dtype=tf.int64, minval=0, maxval=200)

fn_out, _ = sample_transformer(temp_input, temp_target, training=False, 
                               enc_padding_mask=None, 
                               look_ahead_mask=None,
                               dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

TensorShape([64, 36, 8000])